In [528]:
import emoji

emoji_dictionary={
    "0": "\u2764\uFE0F",
    "1": ":baseball:",
    "2": ":grinning_face_with_big_eyes:",
    "3": ":disappointed_face:",
    "4": ":fork_and_knife:",
    "5": ":hundred_points:",
    "6": ":fire:",
    "7": ":face_blowing_a_kiss:",
    "8": ":chestnut:",
    "9": ":flexed_biceps:"

}

In [529]:
import pandas as pd
import numpy as np

In [530]:
train=pd.read_csv('dataset/train_emoji.csv',header=None)
test=pd.read_csv('dataset/test_emoji.csv',header=None)

In [531]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [532]:
data=train.values
print(data.shape)


(131, 4)


In [533]:
X_train=train[0]
Y_train=train[1]

X_test=test[0]
Y_test=test[1]

In [534]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [535]:
f=open('glove.6B.50d.txt',encoding='utf-8')
embeddings_index={}
cnt=0
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float')
    embeddings_index[word]=coefs
f.close()

In [536]:
embeddings_index["eat"]

array([ 6.4295e-01, -4.2946e-01, -5.4277e-01, -1.0307e+00,  1.2056e+00,
       -2.7174e-01, -6.3561e-01, -1.5065e-02,  3.7856e-01,  4.6474e-02,
       -1.3102e-01,  6.0500e-01,  1.6391e+00,  2.3940e-01,  1.2128e+00,
        8.3178e-01,  7.3893e-01,  1.5200e-01, -1.4175e-01, -8.8384e-01,
        2.0829e-02, -3.2545e-01,  1.8035e+00,  1.0045e+00,  5.8484e-01,
       -6.2031e-01, -4.3296e-01,  2.3562e-01,  1.3027e+00, -8.1264e-01,
        2.3158e+00,  1.1030e+00, -6.0608e-01,  1.0101e+00, -2.2426e-01,
        1.8908e-02, -1.0931e-01,  3.8350e-01,  7.7362e-01, -8.1927e-02,
       -3.4040e-01, -1.5143e-03, -5.6640e-02,  8.7359e-01,  1.4805e+00,
        6.9421e-01, -3.0966e-01, -9.0826e-01,  3.7277e-03,  8.4550e-01])

In [537]:
emb_dim=embeddings_index["eat"].shape[0]
print(emb_dim)

50


In [538]:
def embedding_output(X):
    maxLen=10
    embedding_out=np.zeros((X.shape[0],maxLen,emb_dim))
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split()
        for ij in range(len(X[ix])):
            try:
                embedding_out[ix][ij]=embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij]=np.zeros((50,))
    return embedding_out

In [539]:
embeddings_matrix_train=embedding_output(X_train)
embeddings_matrix_test=embedding_output(X_test)

C:\Users\Yash Aryan\AppData\Local\Temp\ipykernel_12896\3145651930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix]=X[ix].split()


In [540]:
print(X_train[0])
print(len(X_train[0]))

['never', 'talk', 'to', 'me', 'again']
5


In [541]:
print(embeddings_matrix_train.shape)
print(embeddings_matrix_test.shape)

(131, 10, 50)
(56, 10, 50)


In [542]:
from keras.utils import to_categorical
Y_train=to_categorical(Y_train,num_classes=5)
Y_test=to_categorical(Y_test,num_classes=5)
print(Y_train.shape)
print(Y_train[0])

(131, 5)
[0. 0. 0. 1. 0.]


In [543]:
from keras.models import Sequential
from keras.layers import *

model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='cetegorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 10, 64)            29440     
                                                                 
 dropout_14 (Dropout)        (None, 10, 64)            0         
                                                                 
 lstm_15 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_15 (Dropout)        (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 5)                 325       
                                                                 
 activation_13 (Activation)  (None, 5)                 0         
                                                                 
Total params: 62,789
Trainable params: 62,789
Non-tra

In [544]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
checkpoint=ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=True,save_best_only=True)
earlystop=EarlyStopping(monitor='val_acc',patience=10)


In [545]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(embeddings_matrix_train, Y_train, epochs=100, batch_size=64, shuffle=True, validation_split=0.2)


Epoch 1/100
2/2 [==============================] - 7s 1s/step - loss: 1.6069 - accuracy: 0.1827 - val_loss: 1.6094 - val_accuracy: 0.2222
Epoch 2/100
2/2 [==============================] - 0s 63ms/step - loss: 1.5714 - accuracy: 0.2404 - val_loss: 1.6081 - val_accuracy: 0.2222
Epoch 3/100
2/2 [==============================] - 0s 63ms/step - loss: 1.5353 - accuracy: 0.3462 - val_loss: 1.6135 - val_accuracy: 0.2222
Epoch 4/100
2/2 [==============================] - 0s 64ms/step - loss: 1.5164 - accuracy: 0.3558 - val_loss: 1.6265 - val_accuracy: 0.1852
Epoch 5/100
2/2 [==============================] - 0s 62ms/step - loss: 1.4922 - accuracy: 0.3942 - val_loss: 1.6447 - val_accuracy: 0.1481
Epoch 6/100
2/2 [==============================] - 0s 63ms/step - loss: 1.4849 - accuracy: 0.3462 - val_loss: 1.6621 - val_accuracy: 0.1852
Epoch 7/100
2/2 [==============================] - 0s 63ms/step - loss: 1.4889 - accuracy: 0.3558 - val_loss: 1.6689 - val_accuracy: 0.2222
Epoch 8/100
2/2 [=====

In [546]:
pred = model.predict(embeddings_matrix_test)
predicted_classes = np.argmax(pred, axis=1)
print(predicted_classes)


2/2 [==============================] - 1s 9ms/step
[4 3 2 2 2 2 3 2 4 2 1 2 0 3 1 3 2 0 3 4 0 3 2 2 3 3 2 0 1 2 0 1 3 2 3 2 2
 3 4 2 1 0 0 2 2 2 2 2 3 3 3 4 3 2 2 3]


In [547]:
model.evaluate(embeddings_matrix_test,Y_test)

2/2 [==============================] - 0s 9ms/step - loss: 2.6487 - accuracy: 0.5714


[2.6486871242523193, 0.5714285969734192]

In [548]:
for i in range(30):
    print(" ".join(X_test[i]))
    print("actual label -", emoji.emojize(emoji_dictionary[str(Y_test[i].argmax())]))
    print("predicted label -", emoji.emojize(emoji_dictionary[str(pred[i].argmax())]))
    print()




I want to eat
actual label - 🍴
predicted label - 🍴

he did not answer
actual label - 😞
predicted label - 😞

he got a raise
actual label - 😃
predicted label - 😃

she got me a present
actual label - ❤️
predicted label - 😃

ha ha ha it was so funny
actual label - 😃
predicted label - 😃

he is a good friend
actual label - ❤️
predicted label - 😃

I am upset
actual label - ❤️
predicted label - 😞

We had such a lovely dinner tonight
actual label - ❤️
predicted label - 😃

where is the food
actual label - 🍴
predicted label - 🍴

Stop making this joke ha ha ha
actual label - 😃
predicted label - 😃

where is the ball
actual label - ⚾
predicted label - ⚾

work is hard
actual label - 😞
predicted label - 😃

This girl is messing with me
actual label - 😞
predicted label - ❤️

are you serious ha ha
actual label - 😃
predicted label - 😞

Let us go play baseball
actual label - ⚾
predicted label - ⚾

This stupid grader is not working
actual label - 😞
predicted label - 😞

work is horrible
actual label - 😞
pred